# AS262 - Projects

# Photometric Redshift Calculation

## Background / Motivation

A photometric redshift is an estimate for the recession velocity of a galaxy made without measuring its spectrum.  Instead the technique uses photometry and broad-band colors to estimate the galaxies redshift.  Using photometric redshifts to estimate the distances of faint galaxies has become an integral part of galaxy surveys conducted during recent years. This is driven by the large number of galaxies and their faint fluxes, which have made spectroscopic follow-up infeasible except for a relatively small and bright fraction of the galaxy population. Albeit less precise and less accurate than spectroscopy, photometric redshifts provide a way to estimate distances for galaxies too faint for spectroscopy or samples too large to be practical for complete spectroscopic coverage.

## Project Outline:
* Use supervised classification to determine the redshift of galaxies based on their broad band photometry.


* The data used for this project will come from the [CEERS surey](https://ceers.github.io), which imaged a region of sky known as the Extended Groth Strip with JWST.  The photometry data is stored in an hdf5 file (similar to the fits table we used in Lecture 3 UVJ exercise, and read in with exactly the same routine, using astropy.table.Table).  This file will be made available on Filer.  Please note that the photometry data has not been published, so it should be considered proprietary for the time being.  I am allowing students to use this data since I am a member of the CEERS collaboration, and you can gain access to it through me.


* You'll most likely want to limit your analysis to relatively bright and/or massive galaxies.  This might mean only working with galaxies whose F356W magnitude is less than 26.5 and/or mass greater than $10^8 M_\odot$.  You should vary these cuts to determine how they affect the final results.


* You’ll need to group the photometry as features in a single `X` array in the format that Scikit-Learn wants and then split the array into a training and test dataset using the `split_samples` routine, which we encountered in Lecture 16. The "known" redshifts come from spectroscopy, given in the table as 'z_spec'.  Note that not all galaxies have spectroscopy, so your training set can only use a subset of the full dataset.  You could perform cross-validation on some subset of this, and then apply your model to the remaining data and compare to the team's photometric redshifts (given as 'ZA_finkelstein' or 'z_phot').


* Try **at least two** supervised classification algorithms and compare their effectiveness by plotting the true redshift against the predicted redshift.  The plot should look something similar to this (although the table I'm providing you doesn't list different quality flags for the spectroscopic redshifts, so all data points would be the same color):


<img src=https://www.colby.edu/physics/faculty/mcgrath/AS262/photoz.png width="500">


* A quantitative measure of how well the classifier is working can be calculated using the Normalized Median Absolute Deviation:

$$ \sigma_{\rm NMAD} = 1.48\times {\rm median}\left(\frac{|\Delta z|}{1+z_{rm true}}\right) $$





* Potential classification algorithms include:
    * Decision Tree Classifier (`sklearn.tree.DecisionTreeRegressor`)
    * Random Forest Classifier (`sklearn.ensemble.RandomForestRegressor`)
    * K-Neighbors Classifier (`sklearn.neighbors.KNeighborsRegressor`)
    * Support Vector Machine (`sklearn.svm.SVR`)

## Example

Here's an example of using a Decision Tree Regression to calculate the photometric redshift of galaxies in the SDSS dataset.

In [ ]:
!pip install astroML

In [ ]:
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
import numpy as np
from matplotlib import pyplot as plt
# import seaborn as sns; sns.set()
%matplotlib inline
%config InlineBackend.figure_format='retina'

from sklearn.tree import DecisionTreeRegressor
from astroML.datasets import fetch_sdss_specgals

#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
if "setup_text_plots" not in globals():
    from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=False)

#------------------------------------------------------------
# Fetch data and prepare it for the computation
data = fetch_sdss_specgals()

In [ ]:
# put magnitudes in a matrix
mag = np.vstack([data['modelMag_%s' % f] for f in 'ugriz']).T
z = data['z']

# train on ~60,000 points
mag_train = mag[::10]
z_train = z[::10]

# test on ~6,000 separate points
mag_test = mag[1::100]
z_test = z[1::100]

In [ ]:
#------------------------------------------------------------
# Compute the cross-validation scores for several tree depths
depth = np.arange(1, 21)
rms_test = np.zeros(len(depth))
rms_train = np.zeros(len(depth))
i_best = 0
z_fit_best = None

for i, d in enumerate(depth):
    clf = DecisionTreeRegressor(max_depth=d, random_state=0)
    clf.fit(mag_train, z_train)

    z_fit_train = clf.predict(mag_train)
    z_fit = clf.predict(mag_test)
    rms_train[i] = np.mean(np.sqrt((z_fit_train - z_train) ** 2))
    rms_test[i] = np.mean(np.sqrt((z_fit - z_test) ** 2))

    if rms_test[i] <= rms_test[i_best]:
        i_best = i
        z_fit_best = z_fit

best_depth = depth[i_best]

In [ ]:
#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(12, 6))
fig.subplots_adjust(wspace=0.25,
                    left=0.1, right=0.95,
                    bottom=0.15, top=0.9)

# first panel: cross-validation
ax = fig.add_subplot(121)
ax.plot(depth, rms_test, '-k', label='cross-validation')
ax.plot(depth, rms_train, '--k', label='training set')
ax.set_xlabel('depth of tree', size=20)
ax.set_ylabel('rms error', size=20)
ax.yaxis.set_major_locator(plt.MultipleLocator(0.01))
ax.set_xlim(0, 21)
ax.set_ylim(0.009,  0.04)
ax.legend(loc=1, fontsize='large')

# second panel: best-fit results
ax = fig.add_subplot(122)
edges = np.linspace(z_test.min(), z_test.max(), 101)
H, zs_bins, zp_bins = np.histogram2d(z_test, z_fit_best, bins=edges)
ax.imshow(H.T, origin='lower', interpolation='nearest', aspect='auto',
           extent=[zs_bins[0], zs_bins[-1], zs_bins[0], zs_bins[-1]],
           cmap=plt.cm.binary)
ax.plot([-0.1, 0.4], [-0.1, 0.4], ':k')
ax.text(0.04, 0.96, "depth = %i\nrms = %.3f" % (best_depth, rms_test[i_best]),
        ha='left', va='top', transform=ax.transAxes, size=12)
ax.set_xlabel(r'$z_{\rm true}$', size=20)
ax.set_ylabel(r'$z_{\rm fit}$', size=20)

ax.set_xlim(-0.02, 0.4001)
ax.set_ylim(-0.02, 0.4001)
ax.xaxis.set_major_locator(plt.MultipleLocator(0.1))
ax.yaxis.set_major_locator(plt.MultipleLocator(0.1))

plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Some preliminaries to get you started using the CEERS dataset:

In [ ]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt

In [ ]:
try:
    photometry_data_path = "/content/drive/MyDrive/Colab Notebooks/ceers_all_v0.51_eazy.hdf5"
    photometry_data_container = Table.read(photometry_data_path)
    # print(photometry_data_container.columns)

except Exception as error_1:
    print(f" { type(error_1).__name__ } occured! ")


def processed_data():

    z_spec = photometry_data_container['z_spec']
    log_mass = photometry_data_container['fast_lmass']
    flux_115_magnitude = -2.5*np.log10(photometry_data_container['FLUX_115'])+31.4 # had conversion aid from Prof McGrath
    flux_150_magnitude = -2.5*np.log10(photometry_data_container['FLUX_150'])+31.4
    flux_200_magnitude = -2.5*np.log10(photometry_data_container['FLUX_200'])+31.4
    flux_277_magnitude = -2.5*np.log10(photometry_data_container['FLUX_277'])+31.4
    flux_356_magnitude = -2.5*np.log10(photometry_data_container['FLUX_356'])+31.4
    flux_410_magnitude = -2.5*np.log10(photometry_data_container['FLUX_410'])+31.4
    flux_444_magnitude = -2.5*np.log10(photometry_data_container['FLUX_444'])+31.4
    flux_606_magnitude = -2.5*np.log10(photometry_data_container['FLUX_606'])+31.4
    flux_814_magnitude = -2.5*np.log10(photometry_data_container['FLUX_814'])+31.4
    flux_125_magnitude = -2.5*np.log10(photometry_data_container['FLUX_125'])+31.4


    # we perform the first series of feature extraction here based on the brightness of the galaxy using flux_356 < 26.5
    flux_356_magnitude = -2.5*np.log10(photometry_data_container['FLUX_356'])+31.4
    mask_from_flux = flux_356_magnitude < 26.5 # this is for the first round of data cleaning.
    # print(mask_for_flux)
    # print(flux_356_magnitude)

    new_z_spec = z_spec[mask_from_flux]
    new_log_mass = log_mass[mask_from_flux]
    new_flux_115_magnitude = flux_115_magnitude[mask_from_flux]
    new_flux_150_magnitude = flux_150_magnitude[mask_from_flux]
    new_flux_200_magnitude = flux_200_magnitude[mask_from_flux]
    new_flux_277_magnitude = flux_277_magnitude[mask_from_flux]
    new_flux_356_magnitude = flux_356_magnitude[mask_from_flux]
    new_flux_410_magnitude = flux_410_magnitude[mask_from_flux]
    new_flux_444_magnitude = flux_444_magnitude[mask_from_flux]
    new_flux_606_magnitude = flux_606_magnitude[mask_from_flux]
    new_flux_814_magnitude = flux_814_magnitude[mask_from_flux]
    new_flux_125_magnitude = flux_125_magnitude[mask_from_flux]


    # now, we perfomr the remainder of the feature extraction based on the z_spec, extracting only features for galaxies with z_spec not equal to -99 or -1.
    # the first feature we'd want to extract is the true spectroscopic redshift z_spec from which we'll discard galaxies with redshits equal to -99 or -1

    mask_from_z_spec = ~( (new_z_spec == -99) | (new_z_spec == -1) )

    desired_z_spec = new_z_spec[mask_from_z_spec]
    desired_log_mass = new_log_mass[mask_from_z_spec]
    desired_flux_115_magnitude = new_flux_115_magnitude[mask_from_z_spec]
    desired_flux_150_magnitude = new_flux_150_magnitude[mask_from_z_spec]
    desired_flux_200_magnitude = new_flux_200_magnitude[mask_from_z_spec]
    desired_flux_277_magnitude = new_flux_277_magnitude[mask_from_z_spec]
    desired_flux_356_magnitude = new_flux_356_magnitude[mask_from_z_spec]
    desired_flux_410_magnitude = new_flux_410_magnitude[mask_from_z_spec]
    desired_flux_444_magnitude = new_flux_444_magnitude[mask_from_z_spec]
    desired_flux_606_magnitude = new_flux_606_magnitude[mask_from_z_spec]
    desired_flux_814_magnitude = new_flux_814_magnitude[mask_from_z_spec]
    desired_flux_125_magnitude = new_flux_125_magnitude[mask_from_z_spec]


    desired_z_phot = (photometry_data_container['z_phot'][mask_from_flux])[mask_from_z_spec]
    desired_z_phot_finkelstein = (photometry_data_container['ZA_finkelstein'][mask_from_flux])[mask_from_z_spec]

    X = np.vstack([desired_log_mass, desired_flux_115_magnitude, desired_flux_150_magnitude, desired_flux_200_magnitude, desired_flux_277_magnitude, desired_flux_356_magnitude, desired_flux_410_magnitude, desired_flux_444_magnitude, desired_flux_606_magnitude, desired_flux_814_magnitude, desired_flux_125_magnitude]).T
    # print(X[:,0].shape)
    y = np.array(desired_z_spec)
    # print(f" The shape of desired_z_spec is {desired_z_spec.shape}")
    # print(f" The shape of desired_z_phot is {desired_z_phot.shape}")
    # print(X)
    # print(y)
    return X, y, desired_z_phot, desired_z_phot_finkelstein


Actual Analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


General code for model classifiers

In [ ]:
X, y, z_phot, ZA_finkelstein = processed_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.45, random_state=42)


Note that this fits the entire data and compares against the teams entire data!

In [ ]:
def compare_to_standard_result(model):
  model.fit(X, y)
  y_pred = model.predict(X)

  figure, ax_main = plt.subplots(1, 2, figsize =(14.5, 7.5))

  ax_main[0].scatter(y_pred, z_phot, color="black")
  ax_main[0].plot([-0.1, 3.5], [-0.1, 3.5], color="green", linestyle="--")
  ax_main[0].set_xlabel(" Model's Prediction ", size=15)
  ax_main[0].set_ylabel(" Z_Phot ", size=15)
  ax_main[0].set_title(" Prediction Against Z_Phot ", size=15)

  ax_main[1].scatter(y_pred, ZA_finkelstein, color="black")
  ax_main[1].plot([-0.1, 3.5], [-0.1, 3.5], color="green", linestyle="--")
  ax_main[1].set_xlabel(" Model's Prediction ", size=15)
  ax_main[1].set_ylabel(" ZA_finkelstein ", size=15)
  ax_main[1].set_title(" Prediction Against ZA_finkelstein ", size=15)

  figure.tight_layout()
  plt.show()

In [ ]:
def model_estimator(model_name, depth_or_neighbor):

  if model_name == KNeighborsRegressor:
    model = model_name(depth_or_neighbor)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    rms_error = np.sqrt(np.mean( (y_test - y_pred)**2 ))
    print(y_pred.shape)

    figure, ax_main = plt.subplots(figsize =(9.5, 7.5))
    ax_main.scatter(y_test, y_pred, label=f" \n neighbors = {depth_or_neighbor} \n  rms error = {rms_error:.4} ", color="black")
    ax_main.plot([-0.1, 3.5], [-0.1, 3.5], color="green", linestyle="--")
    ax_main.set_ylabel(" Predicted Photometric Redshift. ", size=15)
    ax_main.set_xlabel(" True spectroscopic Redshift(z_spec). ", size=15)
    ax_main.set_title(" Predicted photo_z against z_spec. ", size=15)
    ax_main.legend(loc="best")

    figure.tight_layout()
    plt.show()

  else:
     model = model_name(max_depth=depth_or_neighbor, random_state=42)
     model.fit(X_train, y_train)

     y_pred = model.predict(X_test)
     rms_error = np.sqrt(np.mean( (y_test - y_pred)**2 ))
     print(y_pred.shape)

     feature_importance = model.feature_importances_

     figure, ax_main = plt.subplots(1,2,figsize =(14.5, 7.5))
     ax_main[0].scatter(y_test, y_pred, label=f" \n neighbors = {depth_or_neighbor} \n  rms error = {rms_error:.4} ", color="black")
     ax_main[0].plot([-0.1, 3.5], [-0.1, 3.5], color="green", linestyle="--")
     ax_main[0].set_ylabel(" Predicted Photometric Redshift. ", size =15)
     ax_main[0].set_xlabel(" True spectroscopic Redshift(z_spec). ", size =15)
     ax_main[0].set_title(" Predicted photo_z against z_spec. ", size =15 )
     ax_main[0].legend(loc="best")

     ax_main[1].bar(np.arange(len(feature_importance)), feature_importance, color="black")
     ax_main[1].set_ylabel(" Feature Importance ", size = 15)
     ax_main[1].set_xlabel(" Feature Index in X(fitted Data) ", size=15)
     ax_main[1].set_title(" Feature Importance Plot", size =15)

     figure.tight_layout()
     plt.show()


In [ ]:
def cross_validation(model_name):

  max_depth_container = np.arange(1,21)
  train_rms_error = []
  cross_validation_rms_error = []

  for depth in max_depth_container:

    if model_name == KNeighborsRegressor:
      model = model_name(depth, n_jobs = -1)
    elif model_name == DecisionTreeRegressor:
      model = model_name(max_depth=depth, random_state = 42)
    else:
      model = model_name(max_depth=depth, random_state=42, n_jobs=-1)

    model.fit(X_train, y_train)

    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    # print(y_test_pred.shape)

    train_error = np.sqrt(np.mean( (y_train - y_train_pred)**2 ))
    cross_validation_error = np.sqrt(np.mean( (y_test - y_test_pred)**2 ))
    # print(X)

    train_rms_error.append(train_error)
    cross_validation_rms_error.append(cross_validation_error)


  figure, ax_main = plt.subplots(figsize=(10,7.5))

  ax_main.plot(max_depth_container, train_rms_error, label="training error")
  ax_main.plot(max_depth_container, cross_validation_rms_error, label="cross validation error")
  ax_main.set_xlabel("Depth of Tree", size = 15)
  ax_main.set_ylabel("Root Mean Square Error", size =15)
  ax_main.set_title(f" Cross validation for {model_name.__name__}.", size=15)
  ax_main.legend(loc="best", fontsize=10)

  # print(train_rms_error)

Finally the NMAD estimator.

In [ ]:
def NMAD_calculator(model):
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  NMAD = 1.48 * np.median( ( np.abs(y_test - y_pred) )/(1 + y_test) )

  return NMAD

DecisionTreeRegressor

We show the cross validation result for Decision Tree Refgressor.

In [ ]:
cross_validation(DecisionTreeRegressor)


In [ ]:
best_depth_estimate = 17

model_estimator(DecisionTreeRegressor, best_depth_estimate)

Now, we compute the Normalized Median Absolute Deviation for this model.

In [ ]:
decision_tree_model = DecisionTreeRegressor(max_depth=best_depth_estimate, random_state=42)
decision_tree_NMAD = NMAD_calculator(decision_tree_model)

print(f" The Decision Tree Regressor model has an NMAD of {decision_tree_NMAD:.4}.")

Finally we compare this result to the standard results by the CEERS team.

In [ ]:
decision_tree_model = DecisionTreeRegressor(max_depth=best_depth_estimate, random_state=42)

compare_to_standard_result(decision_tree_model)

RandomForestRegressor

In [ ]:
cross_validation(RandomForestRegressor)


In [ ]:
best_depth_estimate = 8

model_estimator(RandomForestRegressor, best_depth_estimate)


Now, we compute the Normalized Median Absolute Deviation value for this model.

In [ ]:
random_forest_model = RandomForestRegressor(max_depth=best_depth_estimate, n_jobs=-1, random_state=42)
random_forest_NMAD = NMAD_calculator(random_forest_model)

print(f" The Random Forest Regressor model has an NMAD of {random_forest_NMAD:.4}.")

Again, we compare the result to the standard result by the CEERS team.

In [ ]:
random_forest_model = RandomForestRegressor(max_depth=best_depth_estimate, n_jobs=-1, random_state=42)

compare_to_standard_result(random_forest_model)

Which is quite excellent indicating much less variability than the prediction of the Decision Tree Regressor Model. So, the Random Forest Regressor model is doing quite a good job.

KNeighborsRegressor

Cross validation for KNeighbor Regressor

In [ ]:
cross_validation(KNeighborsRegressor)

In [ ]:
best_neighbor_estimate = 6
model_estimator(KNeighborsRegressor, best_neighbor_estimate)


Now, we compute the Normalized Median Absolute Deviation for this model.

In [ ]:
kneighbor_model = KNeighborsRegressor(best_neighbor_estimate, n_jobs=-1)
kneighbor_NMAD = NMAD_calculator(kneighbor_model)

print(f" The KNeighbor Regressor model has an NMAD of {kneighbor_NMAD:.4}.")

Again, we make a comparison between the predictions of the KNeighborsRegressor Model and the standard result by the CEERS team.

In [ ]:
kneighbor_model = KNeighborsRegressor(best_neighbor_estimate, n_jobs=-1)

compare_to_standard_result(kneighbor_model)